In [ ]:
# Import thư viện
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns

# Load dữ liệu đã được tiền xử lý
df = pd.read_csv('uber_cleaned.csv')

# 1. Xử lý phân phối của biến mục tiêu
def analyze_and_transform_target(df, target_col='fare_amount'):
    # Vẽ phân phối trước transform
    plt.figure(figsize=(12, 4))
    plt.subplot(121)
    sns.histplot(df[target_col], bins=50)
    plt.title('Phân phối ban đầu của fare_amount')
    
    # Log transform
    df['fare_amount_log'] = np.log1p(df[target_col])
    
    # Vẽ phân phối sau transform
    plt.subplot(122)
    sns.histplot(df['fare_amount_log'], bins=50)
    plt.title('Phân phối sau log transform')
    plt.tight_layout()
    plt.show()
    
    return df

# 2. Feature Engineering nâng cao
def enhanced_feature_engineering(df):
    # Tạo các đặc trưng phi tuyến
    df['trip_distance_squared'] = df['trip_distance'] ** 2
    df['trip_distance_sqrt'] = np.sqrt(df['trip_distance'])
    
    # Tạo các biến tương tác
    df['distance_rush'] = df['trip_distance'] * df['is_rush_hour']
    df['distance_weekend'] = df['trip_distance'] * df['is_weekend']
    
    # Binning cho trip_distance
    df['distance_bin'] = pd.qcut(df['trip_distance'], q=5, labels=['Very Short', 'Short', 'Medium', 'Long', 'Very Long'])
    df = pd.get_dummies(df, columns=['distance_bin'], prefix='dist')
    
    return df

# 3. Xử lý outliers theo nghiệp vụ
def handle_business_outliers(df):
    # Tính giá/km
    df['fare_per_km'] = df['fare_amount'] / df['trip_distance']
    
    # Lọc các chuyến có giá/km hợp lý (giữ lại phạm vi từ 1% đến 99%)
    low_percentile = np.percentile(df['fare_per_km'], 1)
    high_percentile = np.percentile(df['fare_per_km'], 99)
    
    df = df[(df['fare_per_km'] >= low_percentile) & 
            (df['fare_per_km'] <= high_percentile)]
    
    return df

# 4. Xây dựng và đánh giá mô hình
def evaluate_model(y_true, y_pred):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    r2 = r2_score(y_true, y_pred)
    
    print(f'MSE: {mse:.2f}')
    print(f'RMSE: {rmse:.2f}')
    print(f'MAPE: {mape:.2f}%')
    print(f'R²: {r2:.4f}')
    
    return mse, rmse, mape, r2

# 5. Huấn luyện và đánh giá nhiều mô hình
def train_and_evaluate_models(X_train, X_test, y_train, y_test):
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
        'XGBoost': xgb.XGBRegressor(random_state=42),
        'LightGBM': LGBMRegressor(random_state=42)
    }
    
    results = {}
    for name, model in models.items():
        print(f'\nTraining {name}...')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        # Nếu đang dùng log transform, chuyển đổi lại giá trị gốc
        if 'fare_amount_log' in globals():
            y_pred = np.expm1(y_pred)
            y_test_original = np.expm1(y_test)
            metrics = evaluate_model(y_test_original, y_pred)
        else:
            metrics = evaluate_model(y_test, y_pred)
            
        results[name] = metrics
    
    return results

# Main execution
if __name__ == "__main__":
    # 1. Xử lý phân phối
    df = analyze_and_transform_target(df)
    
    # 2. Feature Engineering
    df = enhanced_feature_engineering(df)
    
    # 3. Xử lý outliers
    df = handle_business_outliers(df)
    
    # Chuẩn bị dữ liệu
    feature_cols = ['trip_distance', 'passenger_count', 'hour', 'is_weekend', 
                   'is_rush_hour', 'trip_distance_squared', 'trip_distance_sqrt',
                   'distance_rush', 'distance_weekend'] + \
                  [col for col in df.columns if col.startswith('dist_')]
    
    X = df[feature_cols]
    y = df['fare_amount_log'] if 'fare_amount_log' in df.columns else df['fare_amount']
    
    # Chia tập dữ liệu
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Chuẩn hóa dữ liệu
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Huấn luyện và đánh giá các mô hình
    results = train_and_evaluate_models(X_train_scaled, X_test_scaled, y_train, y_test)

In [ ]:
# 6. Feature Engineering nâng cao hơn nữa
def advanced_feature_engineering(df):
    # Tạo thêm các đặc trưng phi tuyến
    df['trip_distance_cubic'] = df['trip_distance'] ** 3
    df['trip_distance_log'] = np.log1p(df['trip_distance'])
    
    # Time-based features
    df['hour_sin'] = np.sin(2 * np.pi * df['hour']/24)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour']/24)
    
    # Tương tác giữa các features
    df['passenger_distance'] = df['passenger_count'] * df['trip_distance']
    df['rush_hour_passenger'] = df['is_rush_hour'] * df['passenger_count']
    
    # Tạo các nhóm thời gian
    df['time_group'] = pd.cut(df['hour'], 
                            bins=[0, 6, 12, 18, 24], 
                            labels=['Dawn', 'Morning', 'Afternoon', 'Night'])
    df = pd.get_dummies(df, columns=['time_group'], prefix='time')
    
    return df

# Áp dụng feature engineering nâng cao
df = advanced_feature_engineering(df)

In [ ]:
# 7. Hyperparameter Tuning
def tune_random_forest(X_train, y_train):
    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30, None],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }
    
    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(estimator=rf,
                             param_grid=param_grid,
                             cv=5,
                             scoring='neg_mean_squared_error',
                             n_jobs=-1)
    
    grid_search.fit(X_train, y_train)
    print('\nBest parameters:', grid_search.best_params_)
    
    return grid_search.best_estimator_

In [ ]:
# Cập nhật hàm train_and_evaluate_models
def train_and_evaluate_models_v2(X_train, X_test, y_train, y_test):
    # Tune Random Forest
    print('Tuning Random Forest...')
    rf_tuned = tune_random_forest(X_train, y_train)
    
    models = {
        'Linear Regression': LinearRegression(),
        'Random Forest (Tuned)': rf_tuned,
        'XGBoost': xgb.XGBRegressor(random_state=42),
        'LightGBM': LGBMRegressor(random_state=42)
    }
    
    results = {}
    for name, model in models.items():
        print(f'\nTraining {name}...')
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        
        if 'fare_amount_log' in globals():
            y_pred = np.expm1(y_pred)
            y_test_original = np.expm1(y_test)
            metrics = evaluate_model(y_test_original, y_pred)
            
            # Plot residuals
            plt.figure(figsize=(10, 5))
            plt.scatter(y_pred, y_test_original - y_pred, alpha=0.5)
            plt.xlabel('Predicted values')
            plt.ylabel('Residuals')
            plt.title(f'Residual Plot - {name}')
            plt.show()
        
        results[name] = metrics
    
    return results

In [ ]:
# Thử nghiệm mô hình cải tiến
# Chuẩn bị features
features = [
    'trip_distance', 'trip_distance_squared', 'trip_distance_sqrt',
    'trip_distance_cubic', 'trip_distance_log',
    'passenger_count', 'distance_rush', 'distance_weekend',
    'passenger_distance', 'rush_hour_passenger',
    'hour_sin', 'hour_cos',
    'is_weekend', 'is_rush_hour'
] + [col for col in df.columns if col.startswith(('dist_', 'time_'))]

# Chia và chuẩn hóa dữ liệu
X = df[features]
y = df['fare_amount_log']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Huấn luyện và đánh giá các mô hình
results = train_and_evaluate_models_v2(X_train_scaled, X_test_scaled, y_train, y_test)

# So sánh kết quả các mô hình
metrics_df = pd.DataFrame(results).T
print('\nSo sánh các mô hình:')
print(metrics_df)